<a href="https://colab.research.google.com/github/belindalombard/BertMLProject/blob/master/Relevance_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Annotation - BERTimbau - Relevance


---

For this research project, we will be utilizing the BERTimbau pretrained model. See citation below: \

@inproceedings { \
  souza2020bertimbau, \
  author    = {F{\'a}bio Souza and Rodrigo Nogueira and Roberto Lotufo}, \
  title     = {{BERT}imbau: pretrained {BERT} models for {B}razilian {P}ortuguese}, \
  booktitle = {9th Brazilian Conference on Intelligent Systems, {BRACIS}, Rio Grande do Sul, Brazil, October 20-23 (to appear)}, \
  year      = {2020} \
}


## Set Up
Mount to drive to access files for training the model:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

### Libraries

Install transformers:


In [ ]:
!pip install transformers

Import libraries:

In [ ]:
import transformers
from transformers import BertForSequenceClassification, BertTokenizer, pipeline
import pandas as pd
import numpy as np
from torch.utils.data import TensorDataset, random_split, Subset, RandomSampler, SequentialSampler, Dataset
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, classification_report
import os
import torch
from torch.optim import AdamW
import seaborn as sns
import random
import gc

Package versions:

In [ ]:
print(f"Transformers version: {transformers.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Numpy version: {np.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"Scikit-learn version: {sklearn.__version__}")
print(f"Seaborn version: {sns.__version__}")

### Connect to GPU
Connect to GPU if available:

In [ ]:
import torch

# If there's a GPU available:
if torch.cuda.is_available():

    # Use GPU
    device = torch.device("cuda")

    print("There are available GPU(s).") # torch.cuda.device_cour

    print("We will use the GPU:", torch.cuda.get_device_name(0))

# If not

else:
    print("No GPU available, using CPU")
    device = torch.device("cpu")

## GLOBAL VARIABLES - DATA
FILE_PATH = path to data file that will be used \

RELEVANCE = name of column representing relevance \
RELEVANCE_ID = column value that represents (relevance == True) \

POST = name of column representing the content of the tweet \
POST_DATE = name of column representing the post date \
POST_ID = name of column representing the tweet id

YEAR = year of when the dataset is from (to be used for file/folder naming)

In [ ]:
FILE_PATH = '/content/drive/MyDrive/pesquisa-usp/relevant_tweets_2020-12-31.csv'
FOLDER_PATH = '/content/drive/MyDrive/pesquisa-usp/round-3/'

RELEVANCE = 'relevance'
RELEVANCE_ID = 'relevant'

POST = 'content'
POST_DATE = 'post_date'
POST_ID = 'tweet_id'

YEAR = '2020'


## GLOBAL VARIABLES - MODEL

Below are global variables related to training the model. \
This will make the code more maintainable and easier to experiment with new values.

In [ ]:
import os

MODEL_TYPE = 'neuralmind/bert-base-portuguese-cased'
NUM_CLASSES = 2

SEED_VAL = 42

NUM_FOLDS = 5
NUM_EPOCHS = 5

L_RATE = 2e-5 # Testar novos valores
MAX_LEN = 256 # Testar novos valores
BATCH_SIZE = 32 # Testar novos valores
EPS = 1e-8 # Testar novos valores
# regression epsylon

NUM_CORES = os.cpu_count()

## Load BERT

Here, we will load BertForSequenceClassification. \
This is a pre-trained BERT model with a single linear classification layer on top.

In [ ]:
from transformers import BertForSequenceClassification


model = BertForSequenceClassification.from_pretrained(
          MODEL_TYPE,
          num_labels = NUM_CLASSES,
          output_attentions = False,
          output_hidden_states = False
        )


# Send the model to the device:
model.to(device)



---


We will also save the path to this model so that it can be accessed at the start of the training loop.\
Variation of performance was observed when a new BertForSequenceClassification model is loaded.

We plan to compare pre-training performance with post-training performance. \
Thus, we want to ensure that the model being trained is the same model that was previously evaluated.

In [ ]:
model_name = 'original_model.bin'

torch.save(model.state_dict(), model_name)
print('Saved model as', model_name)

## Load Tokenizer

Load BERT tokenizer:

In [ ]:
from transformers import BertTokenizer

print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained(MODEL_TYPE, do_lower_case=False)

## Prepare Dataset

---

Load the cvs file of the dataset chosen to train the model:

In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

df = pd.read_csv(FILE_PATH)

Run this cell to display the dataset:

In [ ]:
df

###  Parsing

Display the class distribution of the dataset:

In [ ]:
print("Tweet_Relevancies", YEAR + ":")

print(df[RELEVANCE].value_counts())

Here, we will recode the class names to use integers instead of strings:

In [ ]:
def condition(x):
    if x == RELEVANCE_ID:
        return 1
    else:
        return 0

df[RELEVANCE] = df[RELEVANCE].apply(condition)


We will illustrate this recoding of class names by displaying the class distributions again:

In [ ]:
print(df[RELEVANCE].value_counts())

Plot overall class distribution and save the graph as a png:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(5, 5))
ax = sns.countplot(df,
                   x=RELEVANCE,
                   hue=RELEVANCE,
                   palette="Paired",
                   width=0.3,
                   legend=False
                  )

plt.title("Class Distribution - Whole Dataset",fontsize=22)
plt.xlabel('Relevance',fontsize=12)
plt.ylabel('Count',fontsize=12)

plt.xticks([0,1],['not relevant', 'relevant'])

plt.grid(True,axis='y',
         linestyle=':',
         linewidth=0.5
        )

for i in ax.containers:
    ax.bar_label(i,)


plt.savefig(FOLDER_PATH+'ClassDistribution_WholeDataset.png',
            format='png',
            transparent = True,
            bbox_inches='tight',
            pad_inches=0.1
           )

### Split Train/Test Sets

Here, we separate **10%** of the total observations for the **test set**. \\
We use train_test_split so that the **data is shuffled** before splitting, using a seed (SEED_VAL) for the random state. \\
We use a stratifier to ensure that **class distribution proportions are maintained** in both sets. \\

**NOTE:** Train and Test sets should be tokenized and processed separately to avoid data leakage. \\

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df,
                                     test_size = 0.1, # 10% for the test set
                                     stratify = df[RELEVANCE], # maintain class proportions
                                     random_state = SEED_VAL # ensures reproducibility of results
                                    )

Display class distributions of each set:

In [ ]:
print("Training Set Class Distribution:")
print(df_train[RELEVANCE].value_counts())
print("\nTest Set Class Distribution:")
print(df_test[RELEVANCE].value_counts())

Here, we plot class distributions across training/test sets and save graph as a png. \
We add the DATASET column for plotting purposes, it will not be relevant later.

In [ ]:
df_train['DATASET'] = 'Train'
df_test['DATASET'] = 'Test'

# Combine the two DataFrames
df_combined = pd.concat([df_train, df_test])

# Plot class distributions in a single plot
plt.figure(figsize=(5, 5))
ax = sns.countplot(x='DATASET',
                   hue=RELEVANCE,
                   data=df_combined,
                   palette="Paired",
                   width=0.5
                  )

plt.title('Class Distribution - Train/Test Sets', fontsize=22)
plt.xlabel('Dataset',fontsize=12)
plt.ylabel('Count',fontsize=12)
plt.legend(title='Relevance', labels=['not relevant', 'relevant'])

plt.grid(True,axis='y',
         linestyle=':',
         linewidth=0.5
        )

for i in ax.containers:
    ax.bar_label(i,)

plt.savefig(FOLDER_PATH+'ClassDistribution_TrainTestSplit.png',
            format='png',
            transparent = True,
            bbox_inches='tight',
            pad_inches=0.1
           )

### Custom Dataset Class

This class will be used to tokenize sentences:

In [ ]:
from torch.utils.data import Dataset

class TweetDataset(Dataset):
  def __init__(self, df):
    self.df_data = df

  def __getitem__(self, index):
    # Get the sentences from the dataframe
    sentence = self.df_data.iloc[index][POST]

    # Transform the sentence

    encoded_dict = tokenizer.encode_plus(sentence,  # Sentences to be tokenized
                                         add_special_tokens = True, # [CLS] and [SEP]
                                         padding = 'max_length',
                                         max_length = MAX_LEN,
                                         return_attention_mask = True, # Build attn. masks.
                                         return_tensors = 'pt', # Return pytorch tensors
                                        )


    # Torch Tensors

    padded_token_list = encoded_dict['input_ids'][0]
    att_mask = encoded_dict['attention_mask'][0]
    token_type_ids = encoded_dict['token_type_ids'][0]


    # Transforming the Target into a tensor

    target = torch.tensor(self.df_data.iloc[index][RELEVANCE])

    sample = (padded_token_list, att_mask, token_type_ids, target)

    return sample


  def __len__(self):

    return len(self.df_data)

### Create Test Dataloader

Here, we create an object of the TweetDataset to tokenize all sentences in our curated dataset. \
We then create a DataLoader object to split the test data into batches, which will be used for the testing loop later one.

In [ ]:
from torch.utils.data import DataLoader

df_test.reset_index(drop=True, inplace=True)
test_data = TweetDataset(df_test)

test_dataloader = DataLoader(test_data,
                             batch_size=BATCH_SIZE,
                             shuffle=False,
                             num_workers=NUM_CORES
                            )

print('Test Dataloader Length:',len(test_dataloader))
print('\nIndex Range of Test Dataloader:', df_test.index)
print('\nFirst 5 Rows of Test Dataloader:')
print(df_test.head())


## Metrics Function

To return evaluation scores, we will use the **classification_report_with_accuracy_score** as defined below.

Arguments: \
- **preds** = a flat list of the predictions of the model as integers
- **labels** = a flat list of the true classification of the corresponding predictions

Returns: \
- **df_class_report** = a DataFrame of the classification report made using **preds** and **labels**, which includes:

 - *precision*, *recall*, and *F1-score* for each class.

 - *macro average* and *weighted average* between both classes for precision, recall and F1-score.


 - **overall_accuracy** = overall *accuracy* of the model.

 All metrics are rounded to the closest 2 decimals.

In [ ]:
from sklearn.metrics import classification_report

def classification_report_with_accuracy_score(preds, labels):
    report = classification_report(labels, preds, output_dict = True, zero_division = 0)

    df_class_report = pd.DataFrame(report).transpose()
    df_class_report = df_class_report.round(decimals=2) # for better readibility

    overall_accuracy = np.sum(preds == labels) / len(labels)
    rounded_accuracy = overall_accuracy.round(decimals=2)

    return df_class_report, rounded_accuracy

## Evaluate the BERTimbau Pre-Trained Model



Here, we will evaluate how BERTimbau (a pre-trained model) performs before being trained on our curated dataset. \
We refer to the process of specifically training BERTimbau on our dataset as fine-tuning, which will be done later on.  

\\
We will evaluate performance using the following metrics: \\
- **Accuracy**: correct predictions/total number of instances
- **Precision**: percentage of instances that the model classified correctly for a given class
- **Recall**: percentage of instances of a given class that were classified correctly
- **F1-Score**: harmonic mean of precision and recall
- **Confusion Matrix**: represents number of actual outputs versus predicted outputs


###Test batches

Use the test_dataloader to evaluate BERTimbau's performance, testing in batches:

In [ ]:
model.eval()
torch.set_grad_enabled(False)

for i, test_batch in enumerate(test_dataloader):
  test_status = 'Batch ' + str(i+1) + ' / ' + str(len(test_dataloader))
  print(test_status, end='\r')

  b_input_ids = test_batch[0].to(device)
  b_input_mask = test_batch[1].to(device)
  b_token_type_ids = test_batch[2].to(device)


  outputs = model(b_input_ids,
                  token_type_ids=None,
                  attention_mask=b_input_mask,
                  return_dict = False)

  preds = outputs[0]
  test_preds = preds.detach().cpu().numpy()


  if i == 0:  # first batch
    stacked_test_preds = test_preds

  else:
    stacked_test_preds = np.vstack((stacked_test_preds, test_preds))


Each prediction returned by the model is a tuple - the logits are still raw.

The first value of the tuple represents the score given for Class 0, and the second value represents the score given for Class 1. \\
To get the real prediction, we take the max of both values to know whether the model's final prediction was Class 0 or Class 1. \\

For clarity, we will illustrate this below:

In [ ]:
print("First 5 predictions as a raw logit tuple:\n", stacked_test_preds[0:6])
final_preds = np.argmax(stacked_test_preds, axis=1)
print("\nFirst 5 predictions final decisions after taking the max:\n", final_preds[0:6])


###Performance Scores

Now, we can calculate the model's performance scores using the function that was previously defined. \

It must be noted that when reloading the model and rerunning the tests, a small fluctuation of values was observed in the performance scores. \
It was ensured that the Train/Test split of the dataset and the tokenization of the test set were the same in all trials.


In [ ]:
labels = df_test[RELEVANCE]

report, accuracy = classification_report_with_accuracy_score(final_preds, labels)

print(report)

Plot report as a table and save figure as a png:

In [ ]:
without_acc = report.loc[['0', '1', 'macro avg', 'weighted avg']]
without_acc = without_acc.rename(index={'0': 'not relevant', '1': 'relevant'})

fig, ax = plt.subplots(figsize=(2,4))
ax.axis('off')
table = pd.plotting.table(ax,
                          without_acc,
                          loc='right',
                          cellLoc='center',
                          rowLoc='center'
                         )

table.auto_set_font_size(False)
table.set_fontsize(14)
table.scale(2.9, 3)
plt.title('BERTimbau Performance Before Fine-Tuning',
          loc='left',
          fontsize=20
         )

acc_str = 'Overall Accuracy: '+ str(accuracy)
fig.text(0.67, 0.08,
         acc_str,
         ha='center',
         fontsize=14
        )


plt.savefig(FOLDER_PATH+'PerformanceTable_PreTrainedModel.png',
            format='png',
            transparent = True,
            bbox_inches='tight',
            pad_inches=0.1
           )

Plot the confusion matrix to visualize the model's predictions against their true value, and save the figure as a png:

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns


confusion_matrix = confusion_matrix(df_test[RELEVANCE], final_preds)

fig = plt.figure(figsize=(10, 8))

ax = sns.heatmap(confusion_matrix,
                 annot=True,
                 cmap="rocket_r",
                 fmt = '.0f',
                 yticklabels=['not relevant', 'relevant'],
                 xticklabels=['not relevant', 'relevant'],
                 square=True,
                 annot_kws={"size": 14},
                 cbar=True,
                 cbar_kws={'shrink': 0.9}
                )

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Add title and labels
plt.title('BERTimbau Performance Before Fine-Tuning',
          fontsize = 22,
          loc = 'center',
          pad = 20
         )

plt.xlabel('Predicted Labels', fontsize=16)
plt.ylabel('True Labels', fontsize=16)

plt.savefig(FOLDER_PATH+'ConfusionMatrix_PreTrainedModel.png',
            format='png',
            transparent = True,
            bbox_inches='tight',
            pad_inches=0.1
           )


##Train Model

We will now train BERTimbau on our curated dataset.\
The goal is to achieve significantly better evaluation scores compared to the pre-trained model's performance.

###Set the Seed

The seed will ensure reproducibility of these experiments:

In [ ]:
random.seed(SEED_VAL)
np.random.seed(SEED_VAL)
torch.manual_seed(SEED_VAL)
torch.cuda.manual_seed_all(SEED_VAL)

###Remove Warnings

In [ ]:
from transformers import AdamW
import random
import gc
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import shuffle

# Remove Warnings

from transformers import logging

logging.set_verbosity_warning()

###Create Folds

In order to train the model, we must split the training dataset into train/validation sets.\
To do this, we will create folds such that each fold will have a different randomized (but seed controlled) dataset split.

In [ ]:
skf = StratifiedKFold(n_splits = NUM_FOLDS,
                     shuffle = True,
                     random_state = SEED_VAL)

targets = df_train[RELEVANCE]

folds_list = skf.split(df_train,targets)

###Class Distribution Across Folds

In [ ]:
# create a figure with subplots
fig, axes = plt.subplots(1,NUM_FOLDS, figsize=(18, 8))
fig.suptitle('Class Distributions Across Folds', fontsize=18)

for n, (train_index, val_index) in enumerate(skf.split(df_train,targets)):
  train = df_train.iloc[train_index,:]
  validation = df_train.iloc[val_index,:]

  # Group the datasets to be plotted:
  train.loc[:, 'DATASET'] = 'Train'
  validation.loc[:, 'DATASET'] = 'Validation'
  df_combined = pd.concat([train, validation])
  # plt.figure(figsize=(5, 5))

  sub_fig = axes[n]
  ax = sns.countplot(ax=sub_fig, x='DATASET', hue=RELEVANCE, data=df_combined, palette="Paired", width=0.5)

  title = 'Fold ' + str(n+1)
  sub_fig.set_title(title, fontsize=14)

  # Set labels and legend
  sub_fig.set_xlabel('Dataset', fontsize=12)
  sub_fig.set_ylabel('')
  sub_fig.legend(title='Relevance', loc='upper right')
  sub_fig.grid(True, axis='y', linestyle=':', linewidth=0.5)


  for i in ax.containers:
      ax.bar_label(i,)


plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(FOLDER_PATH+'ClassDistribution_AcrossFolds.png', format='png', transparent = True, bbox_inches='tight',pad_inches=0.1)

###Lists for Performance Scores/Data

We will use the list below to store performance scores of all epochs across all folds.

Each epoch is a dictionary containing:
- fold number
- epoch number

- total train loss
- average train loss
- total validation loss
- average validation loss

- classification report
- validation accuracy

In [ ]:
training_stats = []

###Training Loop

We train the model in NUM_FOLDS folds, each of which will have NUM_EPOCHS epochs.


Each fold loads a new pre-trained BERTimbau model to fine-tune. \

Each epoch in this fold will train the model on the fold's train set, followed by an evaluation of the model using the fold's validation set. \
\
If its Macro F1-score is better than the other epochs, that model will be saved using: \
  - model_name = 'fold_' + str(fold_index) + '.bin'
  - torch.save(model.state_dict(), model_name)

We will also use a variable to store the name of the best model out of all folds, so that we can load it later for testing.






In [ ]:
# best_fold_path = ''

In [ ]:
%%time

for fold_index, (train_index, val_index) in enumerate(skf.split(df_train,targets)):
  print('\n################## Fold Model Number', str((fold_index+1)) + ' / ' + str(NUM_FOLDS), '##################')

  # ..................................................
  # Defining Dataloaders of Training and Validation
  # ..................................................

  # Train and Validation Sets
  train = df_train.iloc[train_index,:]
  val = df_train.iloc[val_index,:]


  # Reseting Indices so it will work
  train = train.reset_index(drop = True)
  val = val.reset_index(drop = True)


  # Tokenizing and Transforming Inputs (att masks, labels, etc.)
  train_data = TweetDataset(train)
  val_data = TweetDataset(val)

  # Train/Validation Split for this Fold:
  train_dataloader = torch.utils.data.DataLoader(train_data,
                                                 batch_size=BATCH_SIZE,
                                                 shuffle=False,
                                                 num_workers=NUM_CORES)

  val_dataloader = torch.utils.data.DataLoader(val_data,
                                               batch_size=BATCH_SIZE,
                                               shuffle=False,
                                               num_workers=NUM_CORES)

  # ..................................................
  # List for Epoch Evaluation Metric:
  # ..................................................
  fold_f1_macro_list = []

#------------------------------------------------------------------------------
  for epoch in range(NUM_EPOCHS):
#------------------------------------------------------------------------------

    #......................
    # Loading Fold Model
    #......................

    if epoch == 0:

      # Load model from pre-training evaluation:
      model.load_state_dict(torch.load('original_model.bin'))

      optimizer = AdamW(model.parameters(),
                        lr = L_RATE,
                        eps = EPS
                       )

    else:
      # Load the fold's model:
      path_model = 'fold_' + str(fold_index) + '.bin'

      if os.path.exists(path_model):
        model.load_state_dict(torch.load(path_model))

      else:
        print(f"Error: {path_model} does not exist")

        model.to(device)


    # ==========================================================================
    #               Training
    # ==========================================================================
    print('============== Epoch {:} / {:} =============='.format(epoch + 1, NUM_EPOCHS))
    print(" ")
    print('\nTraining...')
    print('')

    # Put the model into train mode:
    model.train()

    # Turn gradient calculations on:
    torch.set_grad_enabled(True)


    # .......................................................
    # Defining Variables to Track This Epoch's Performance
    # .......................................................

    # overall stats:
    epoch_stats = {'fold': fold_index + 1,
                   'epoch': epoch + 1}

    total_macro_f1 = 0
    avg_macro_f1 = 0

    # train stats:
    total_train_loss = 0
    avg_train_loss = 0

    # validation stats:
    total_val_loss = 0
    avg_val_loss = 0

    # ..................
    # Train the Model
    # ..................

    for i, batch in enumerate(train_dataloader):

      train_status = 'Batch ' + str(i+1) + ' of ' + str(len(train_dataloader))

      print(train_status, end='\r')


      b_input_ids = batch[0].to(device)
      b_input_mask = batch[1].to(device)
      b_token_type_ids = batch[2].to(device)
      b_labels = batch[3].to(device)

      model.zero_grad()


      loss, logits = model(b_input_ids,
                           token_type_ids=None,
                           attention_mask=b_input_mask,
                           return_dict = False,
                           labels=b_labels)

      # Calculate loss:
      batch_loss = loss.item()

      total_train_loss += batch_loss


      # Zero the gradients:
      optimizer.zero_grad()

      # Perform a backward pass to calculate the gradients:
      loss.backward()


      # Clip the norm of the gradients to 1.0.
      # This is to help prevent the "exploding gradients" problem.
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)


      # Update Weights:
      optimizer.step()   # for GPU

      # https://pytorch.org/xla/
      #xm.optimizer_step(optimizer, barrier=True)   #for TPU


    # ........................................
    # Update Training Performance Variables
    # ........................................

    avg_train_loss = total_train_loss / len(train_dataloader)

    print('Total Train loss:' ,total_train_loss)
    print('Average Train Loss: ', avg_train_loss)


    # .......................................
    # Add Scores to Epoch Stats Dictionary
    # .......................................

    epoch_stats['total_train_loss'] = total_train_loss
    epoch_stats['avg_train_loss'] = avg_train_loss



    # ==========================================================================
    #               Validation
    # ==========================================================================
    print(" ")
    print('=============================================')
    print('\nValidation...')
    print(' ')

    # Put the model in evaluation mode:
    model.eval()

    # Turn off the gradient calculations.
    # This tells the model not to compute or store gradients.
    # This step saves memory and speeds up validation.
    torch.set_grad_enabled(False)


    # Lists to accumulate all predictions and their corresponding labels:
    stacked_val_preds = []
    val_targets = []


    # .....................
    # Validate the Model
    # .....................

    for j, val_batch in enumerate(val_dataloader):

      val_status = 'Batch ' + str(j+1) + ' of ' + str(len(val_dataloader))

      print(val_status, end='\r')


      b_input_ids = val_batch[0].to(device)
      b_input_mask = val_batch[1].to(device)
      b_token_type_ids = val_batch[2].to(device)
      b_labels = val_batch[3].to(device)


      loss, logits = model(b_input_ids,
                           token_type_ids=None,
                           attention_mask=b_input_mask,
                           return_dict = False,
                           labels=b_labels)


      # Calculate loss:
      batch_loss = loss.item()
      total_val_loss += batch_loss


      # Move labels and predictions to the CPU:
      batch_targets = b_labels.to('cpu').numpy()
      batch_preds = logits.detach().cpu().numpy()


      # Append the labels and predictions to their lists:
      val_targets.extend(batch_targets)

      if j == 0:
        stacked_val_preds = batch_preds
      else:
        stacked_val_preds = np.vstack((stacked_val_preds, batch_preds))


      batch_preds = np.argmax(batch_preds, axis = 1)

      total_macro_f1 += f1_score(batch_targets,
                                 batch_preds,
                                 average = 'macro'
                                )
      # usage: sklearn.metrics.f1_score(y_true, y_pred)


    # ..........................................
    # Update Validation Performance Variables
    # ..........................................

    avg_val_loss = total_val_loss / len(val_dataloader)

    avg_macro_f1 = total_macro_f1 / len(val_dataloader)


    print('Total Validation loss:' ,total_val_loss)
    print('Average Validation Loss: ', avg_val_loss)

    print("Macro F1-score: {0:.2f}".format(avg_macro_f1))



    stacked_val_preds = np.argmax(stacked_val_preds, axis=1)

    class_report, val_acc = classification_report_with_accuracy_score(stacked_val_preds, val_targets)


    print('Validation accuracy: ', val_acc)
    print('Classification Report: \n', class_report)



    # .......................................
    # Add Scores to Epoch Stats Dictionary
    # .......................................
    epoch_stats['total_val_loss'] = total_val_loss
    epoch_stats['avg_val_loss'] = avg_val_loss
    epoch_stats['val_accuracy'] = val_acc
    epoch_stats['report'] = class_report

    # ......................
    # Save the best model
    # ......................

    if epoch == 0:

      # Save the Model:
      model_name = 'fold_' + str(fold_index) + '.bin'
      torch.save(model.state_dict(), model_name)
      print('Saved model as', model_name)

    else:


      # Get the best Macro-F1 score seen so far:
      best_f1_macro = max(fold_f1_macro_list)

      # Keep the model with the best Macro-F1 score:
      if (avg_macro_f1 > best_f1_macro) and (avg_val_loss <= avg_train_loss):

        model_name = 'fold_' + str(fold_index) + '.bin'
        torch.save(model.state_dict(), model_name)

        # best_fold_path = model_name
        print('F1-Macro improved. Saved model as', model_name)


        epoch_stats['f1_macro'] = avg_macro_f1


    # Add the Macro-F1 scores of this epoch to the overall list:
    fold_f1_macro_list.append(avg_macro_f1)

    # .........................................
    # Add Epoch Stats to Training Stats List
    # .........................................
    training_stats.append(epoch_stats)


    # Use the garbage collector to save memory:
    gc.collect()


##Model Evaluation

### Prepare Data

Convert the training_stats dictionary into a dataframe that will be used for data visualization:

In [ ]:
pd.set_option('display.precision', 2)

df = pd.DataFrame(training_stats)


Save the data frame as cvs and excel files:

In [ ]:
path = FOLDER_PATH + 'training_stats.cvs'
df.to_csv(path)

path = FOLDER_PATH + 'training_stats.xlsx'
df.to_excel(path)

Display the dataframe to see what it looks like:

In [ ]:
df

### Compare Folds

Group the training stats dataframe by folds and filter out losses columns:




In [ ]:
folds_loss = df.groupby('fold')[['total_train_loss','avg_train_loss','total_val_loss', 'avg_val_loss']].mean()
folds_loss = folds_loss.reset_index()

In [ ]:
folds_loss

Plot the losses of each fold:

In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(x='fold', y='total_train_loss', data=folds_loss, marker='o', label='Total Train Loss')
sns.lineplot(x='fold', y='avg_train_loss', data=folds_loss, marker='o', label='Average Train Loss')
sns.lineplot(x='fold', y='total_val_loss', data=folds_loss, marker='o', label='Total Val Loss')
sns.lineplot(x='fold', y='avg_val_loss', data=folds_loss, marker='o', label='Average Val Loss')


plt.title('Training and Validation Losses Across Folds', fontsize=20)

plt.xlabel('Fold',fontsize=16)
plt.xticks(folds_loss['fold'], fontsize=12)
plt.ylabel('Loss',fontsize=16)
plt.yticks(fontsize=12)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left',fontsize = 12)
plt.grid(axis='y')

plt.savefig(FOLDER_PATH+'Training_Validation_Losses_Across_Folds.png', format='png', transparent = True, bbox_inches='tight',pad_inches=0.1)

Group the training stats data frame by folds and filter out the validation accuracy column:

In [ ]:
folds_acc = df.groupby('fold')['val_accuracy'].mean()
folds_acc = folds_acc.reset_index()

In [ ]:
folds_acc

Plot validation accuracy across folds:

In [ ]:
plt.figure(figsize=(10, 6))

sns.lineplot(x='fold', y='val_accuracy', data=folds_acc, marker='o')


plt.title('Validation Accuracy Across Folds',fontsize=20)

plt.xlabel('Fold', fontsize = 16)
plt.xticks(folds_acc['fold'],fontsize=12)
plt.ylabel('')
plt.yticks([50,55,60,65,70,75,80,85,90,95,100],fontsize=12)
plt.grid(axis='y')

plt.savefig(FOLDER_PATH+'Validation_Accuracy_Across_Folds.png', format='png', transparent = True, bbox_inches='tight',pad_inches=0.1)

Plot f1-macro scores across folds:

In [ ]:
folds_f1 = df.groupby('fold')['f1_macro'].mean()
folds_f1 = folds_f1.reset_index()

In [ ]:
folds_f1

In [ ]:
plt.figure(figsize=(10, 6))

sns.lineplot(x='fold', y='macro f1-score', data=folds_acc, marker='o')


plt.title('Macro F1-Score Across Folds',fontsize=20)

plt.xlabel('Fold', fontsize = 16)
plt.xticks(folds_f1['fold'],fontsize=12)
plt.ylabel('')
plt.yticks([50,55,60,65,70,75,80,85,90,95,100],fontsize=12)
plt.grid(axis='y')

plt.savefig(FOLDER_PATH+'Validation_Accuracy_Across_Folds.png', format='png', transparent = True, bbox_inches='tight',pad_inches=0.1)

### Compare Epochs of The Best Fold

In [ ]:
folds_f1 = df.groupby('fold')['f1_macro'].mean()
best = folds_f1.idxmax()
best

In [ ]:
folds_f1

In [ ]:
epochs = df[df['fold'] == best].groupby('epoch')[['total_train_loss','avg_train_loss','total_val_loss', 'avg_val_loss']].mean()
epochs = epochs.reset_index()
epochs

Compare epoch losses of the best fold:

In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(x='epoch', y='total_train_loss', data=epochs, marker='o', label='Total Train Loss')
sns.lineplot(x='epoch', y='total_val_loss', data=epochs, marker='o', label='Total Val Loss')

plt.title('Total Losses Across Epochs', fontsize=20)

plt.xlabel('Epoch',fontsize=16)
plt.xticks(epochs['epoch'],fontsize=12)
plt.ylabel('Loss',fontsize=16)
plt.yticks(fontsize=12)

plt.legend(loc='upper right',fontsize = 12)
plt.grid(axis='y')

plt.savefig(FOLDER_PATH+'Total_Losses_Across_Epochs.png', format='png', transparent = True, bbox_inches='tight',pad_inches=0.1)

In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(x='epoch', y='avg_train_loss', data=epochs, marker='o', label='Average Train Loss')
sns.lineplot(x='epoch', y='avg_val_loss', data=epochs, marker='o', label='Average Val Loss')


plt.title('Average Losses Across Epochs', fontsize=20)

plt.xlabel('Epoch',fontsize=16)
plt.xticks(epochs['epoch'],fontsize=12)
plt.ylabel('Loss',fontsize=16)
plt.yticks(fontsize=12)

plt.legend(loc='upper right',fontsize = 12)
plt.grid(axis='y')

plt.savefig(FOLDER_PATH+'Average_Losses_Across_Epochs.png', format='png', transparent = True, bbox_inches='tight',pad_inches=0.1)

### Evaluate the Best Fold on Test Set

Here, we evaluate the trained model of the best fold, using the same method used to evaluate the pre-trained model. \
We do this to see if there was significant improvement after being trained with the curated dataset.

In [ ]:
best_fold_path = 'fold_' + str(best) + '.bin'
best_fold_path

In [ ]:
# Load best model:
if os.path.exists(best_fold_path):
    model.load_state_dict(torch.load(best_fold_path))

else:
  print(f"Error: {best_fold_path} does not exist")

####Test Batches

In [ ]:
model.eval()
torch.set_grad_enabled(False)

for i, test_batch in enumerate(test_dataloader):
  test_status = 'Batch ' + str(i+1) + ' / ' + str(len(test_dataloader))
  print(test_status, end='\r')

  b_input_ids = test_batch[0].to(device)
  b_input_mask = test_batch[1].to(device)
  b_token_type_ids = test_batch[2].to(device)


  outputs = model(b_input_ids,
                  token_type_ids=None,
                  attention_mask=b_input_mask,
                  return_dict = False)

  preds = outputs[0]
  test_preds = preds.detach().cpu().numpy()


  if i == 0:  # first batch
    stacked_test_preds = test_preds

  else:
    stacked_test_preds = np.vstack((stacked_test_preds, test_preds))


# Round predictions to either 0 or 1
final_preds = np.argmax(stacked_test_preds, axis=1)

####Performance Scores

Like before, we can calculate the model's performance scores using the function that was previously defined. \

In [ ]:
labels = df_test[RELEVANCE]

report, accuracy = classification_report_with_accuracy_score(final_preds, labels)
# report is a DataFrame
print(report)

Get the missclassified tweets:

In [ ]:
posts = df_test[POST]
ids = df_test[POST_ID]

In [ ]:
posts

In [ ]:
ids

In [ ]:
wrong_preds = []

for i in range(len(labels)):
  predicted = final_preds[i]
  correct = labels[i]

  if predicted != label:
    info = {'post id': ids[i],
            'post content': posts[i],
            'prediction': predicted,
            'correct label': correct}
    wrong_preds.append(info)

In [ ]:
df = pd.DataFrame(wrong_classification)
df

In [ ]:
path = FOLDER_PATH + 'wrong_predictions.cvs'
df.to_csv(path)

path = FOLDER_PATH + 'wrong_predictions.xlsx'
df.to_excel(path)

Plot report as a table and save figure as a png:

In [ ]:
without_acc = report.loc[['0', '1', 'macro avg', 'weighted avg']]
without_acc = without_acc.rename(index={'0': 'not relevant', '1': 'relevant'})

fig, ax = plt.subplots(figsize=(2,4))
ax.axis('off')
table = pd.plotting.table(ax, without_acc, loc='right',cellLoc='center',rowLoc='center')
table.auto_set_font_size(False)
table.set_fontsize(14)
table.scale(2.9, 3)
plt.title('BERTimbau Performance After Fine-Tuning',
           loc='left',fontsize=20)
acc_str = 'Overall Accuracy: '+ str(accuracy.round(decimals=2))
fig.text(0.67, 0.08, acc_str, ha='center', fontsize=14)

plt.savefig(FOLDER_PATH+'PerformanceTable_TrainedModel.png', format='png', transparent = True, bbox_inches='tight',pad_inches=0.1)

Plot the confusion matrix to visualize the model's predictions against their true value, and save the figure as a png:

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns


confusion_matrix = confusion_matrix(df_test[RELEVANCE], final_preds)
fig = plt.figure(figsize=(10, 8))
ax = sns.heatmap(confusion_matrix, annot=True,cmap="rocket_r", fmt = '.0f',
            yticklabels=['not relevant', 'relevant'],
            xticklabels=['not relevant', 'relevant'],square=True,
                 annot_kws={"size": 14},cbar=True,
                 cbar_kws={'shrink': 0.9})
# ax.set_font
# plt.yticks(rotation=0)
plt.xticks(fontsize=12)  # X-axis tick labels font size
plt.yticks(fontsize=12)
# Add title and labels
plt.title('BERTimbau Performance After Fine-Tuning', fontsize=22, loc='center',
          pad=20)
plt.xlabel('Predicted Labels', fontsize=16)
plt.ylabel('True Labels', fontsize=16)

plt.savefig(FOLDER_PATH+'ConfusionMatrix_TrainedModel.png', format='png', transparent = True, bbox_inches='tight',pad_inches=0.1)
